## Sequence Generator

In [23]:
from random import randint
from numpy import array
from numpy import argmax
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):    
    return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [24]:
# generate random sequence
sequence = generate_sequence()
print(sequence)
# one hot encode
encoded = one_hot_encode(sequence)
print(encoded)
# one hot decode
decoded = one_hot_decode(encoded)
print(decoded)

[25, 91, 83, 89, 69, 53, 28, 57, 75, 35, 0, 97, 20, 89, 54, 43, 35, 19, 27, 97, 43, 13, 11, 48, 12]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[25, 91, 83, 89, 69, 53, 28, 57, 75, 35, 0, 97, 20, 89, 54, 43, 35, 19, 27, 97, 43, 13, 11, 48, 12]


## Sequence -> LSTM Features

In [25]:
# generate data for the lstm
def generate_data():
    # generate sequence
    sequence = generate_sequence()

    # one hot encode
    encoded = one_hot_encode(sequence)

    # convert to 3d for input
    # LSTMs require a 3D input:
    #    [samples, timesteps, features].    
    X = encoded.reshape(encoded.shape[0], 1, encoded.shape[1])
    return sequence, X, encoded

In [30]:
sequence, X, encoded = generate_data()
print('sequence', sequence)
print('sequence length:', len(sequence))

# 25 samples per sequence, 1 time step, and 100 one-hot features
print('X shape:', X.shape)

print('one-hot-encoded shape:', encoded.shape)

print('X[0]', X[0])
print('encoded[0]', encoded[0])

sequence [19, 80, 20, 87, 54, 76, 8, 49, 48, 76, 59, 67, 32, 70, 1, 87, 92, 14, 87, 68, 96, 34, 98, 82, 43]
sequence length: 25
X shape: (25, 1, 100)
one-hot-encoded shape: (25, 100)
X[0] [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
encoded[0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Part 1: LSTM echoer

Train a trivial LSTM to predict the current observation $X(t)$

$$\hat{y}(t) = f(X(t))$$

Where
- $f(X(t))$ is the trained LSTM model that echos the current observation
- $\hat{y}(t)$ is the one-hot encoded representation of X

Once this is done, we can re-train that LSTM to predict current observations based on previous observations.

In [39]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

n_memory_units = 15
n_time_steps = 1
n_features = 100

# input features: 1 step, 100
# output features: 100
#
# categorical_crossentropy: y is one-hot
#    (similar to multi-class classification)
# softmax: emit probabilities that sum up to 1

model = Sequential()
model.add(LSTM(n_memory_units, input_shape=(n_time_steps, n_features)))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 15)                6960      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               1600      
Total params: 8,560
Trainable params: 8,560
Non-trainable params: 0
_________________________________________________________________


In [40]:
# fit model on 500 random sequences
for i in range(500):
    _, X, y = generate_data()
    model.fit(X, y, epochs=1, batch_size=1, verbose=1)

Epoch 1/1
25/25 [==============================] - 1s 29ms/step - loss: 4.6036 - acc: 0.0800
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.5992 - acc: 0.0400
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.6047 - acc: 0.0800
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.5975 - acc: 0.0400
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 4.5974 - acc: 0.0000e+00
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.6042 - acc: 0.0000e+00
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.5926 - acc: 0.0800
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.6041 - acc: 0.0000e+00
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 4.6076 - acc: 0.0400
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 4.5961 - acc: 0.1200
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - lo

25/25 [==============================] - 0s 2ms/step - loss: 3.3714 - acc: 0.8000
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 3.0549 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 3.1056 - acc: 0.9600
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 3.1958 - acc: 0.8800
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 3.1759 - acc: 0.8800
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 3.1847 - acc: 0.8000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 3.0070 - acc: 0.8000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 2.8708 - acc: 0.9200
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 2.9460 - acc: 0.8800
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 2.8747 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 2.8445 - acc: 0.920

25/25 [==============================] - 0s 2ms/step - loss: 1.4186 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.4238 - acc: 0.9600
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 1.5078 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.4003 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.3919 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.5774 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.6020 - acc: 0.9600
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.2847 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.3221 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.4008 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 1.4095 - acc: 1.000

25/25 [==============================] - 0s 4ms/step - loss: 0.7015 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.6213 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - ETA: 0s - loss: 0.5763 - acc: 1.000 - 0s 3ms/step - loss: 0.6176 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.6054 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 5ms/step - loss: 0.5344 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.6005 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.5553 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.5606 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.5785 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.5430 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0

25/25 [==============================] - 0s 2ms/step - loss: 0.2779 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2607 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 0.3120 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 2ms/step - loss: 0.2476 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2657 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2712 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2907 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2925 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2393 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2676 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.2958 - acc: 1.000

25/25 [==============================] - 0s 4ms/step - loss: 0.1236 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.1413 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1127 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1459 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1406 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.1298 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1112 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1291 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 3ms/step - loss: 0.1326 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1206 - acc: 1.0000
Epoch 1/1
25/25 [==============================] - 0s 4ms/step - loss: 0.1318 - acc: 1.000

In [44]:
# evaluate model on new data
sequence, X, y = generate_data()
yhat = model.predict(X)
print('Sequence: %s' % sequence)
print('Predicted (one-hot): %s' % yhat)
print('Predicted (decoded): %s' % one_hot_decode(yhat))

Sequence: [51, 33, 38, 68, 17, 20, 97, 15, 57, 45, 66, 65, 94, 29, 73, 56, 64, 47, 56, 1, 69, 22, 7, 3, 33]
Predicted (one-hot): [[1.2238875e-05 1.1723145e-04 2.1980189e-02 ... 1.2829198e-04
  4.2110114e-04 8.4455364e-04]
 [9.8028465e-04 1.2537774e-04 4.0743533e-05 ... 4.0788593e-05
  2.2807892e-03 1.5228044e-03]
 [6.2082654e-05 1.5232925e-05 6.7704565e-07 ... 4.1883797e-04
  2.5956685e-04 1.6445362e-04]
 ...
 [1.2205047e-05 1.0759633e-05 3.0804430e-03 ... 2.0842951e-04
  7.3193456e-04 9.1206733e-05]
 [6.0820145e-07 3.8882521e-05 1.5315751e-04 ... 1.4556943e-04
  7.4113888e-04 4.1841715e-05]
 [9.8028465e-04 1.2537774e-04 4.0743533e-05 ... 4.0788593e-05
  2.2807892e-03 1.5228044e-03]]
Predicted (decoded): [51, 33, 38, 68, 17, 20, 97, 15, 57, 45, 66, 65, 94, 29, 73, 56, 64, 47, 56, 1, 69, 22, 7, 3, 33]


## Part 2: LSTM predictor

Next, we'll use the same LSTM model to predict the current observation
based on history (previous observations).

$$ \hat{y}(t) = f(X(t-n), ..., X(t-1))$$

Where
- $n$ is the number of lag time steps
- $f(X(t-n))$ is the trained LSTM model that predicts the current observation
- $\hat{y}(t)$ is the one-hot encoded representation of the current observation (time t)

Changes needed:

- Convert dataset into a lagged dataset
- Ensure that the LSTM is setup to store state (`stateful=True`)